In [2]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json


requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

url = "https://bdtd.ibict.br/vufind/api/v1/search"

In [4]:
stats = requests.get(url + "?lookfor=*%3A*&type=AllFields&facet[]=format&facet[]=institution&limit=0&filter[]=institution%3AUFRJ", verify=False).json()

print(f"Documentos: {stats['resultCount']}")
for facet in stats["facets"]["format"]:
    print(f"{facet['translated']}: {facet['count']}")

Documentos: 2832
Dissertação: 2205
Tese: 627


In [20]:
def get_json_data(metadata, n=100, fp=10):
    count = metadata["count"]
    records = []
    i = 0
    for i in range(0, count//n):
        current_url = url + metadata["href"].replace("limit=0", f"limit={n}") + f"&page={i+1}"
        req = requests.get(current_url, verify=False).json()
        print(req)
        records += req["records"]
        if i % fp == 0:
            print(current_url)
    if (count//n)*n < count:
        i += 1
        print("Last Index:", i+1)
        current_url = url + metadata["href"].replace("limit=0", f"limit={n}") + f"&page={i+1}"
        print("Last URL:", current_url)
        req = requests.get(current_url, verify=False).json()
        print(req)
        records += req["records"]
        
    return records

In [12]:
facets = stats["facets"]["format"]
facets


[{'value': 'masterThesis',
  'translated': 'Dissertação',
  'count': 2205,
  'href': '?limit=0&filter%5B%5D=institution%3A%22UFRJ%22&filter%5B%5D=format%3A%22masterThesis%22&lookfor=%2A%3A%2A&type=AllFields'},
 {'value': 'doctoralThesis',
  'translated': 'Tese',
  'count': 627,
  'href': '?limit=0&filter%5B%5D=institution%3A%22UFRJ%22&filter%5B%5D=format%3A%22doctoralThesis%22&lookfor=%2A%3A%2A&type=AllFields'}]

In [21]:
# Doutorados:
metadata = facets[1]
count = metadata["count"]
thesis_metadata = get_json_data(metadata, fp=20)
print(count, len(thesis_metadata))
json_object = json.dumps(thesis_metadata, indent = 4) 

with open("doutorados.json", "w") as f:
    f.write(json_object) 

{'resultCount': 627, 'records': [{'id': 'UFRJ_3893129d6c54ddd7e1bb9b31d43dd4ba', 'title': 'Influência das oscilações subinerciais na variabilidade de temperatura na plataforma continental sudeste do Brasil', 'authors': {'primary': {'França, Bruna Reis Leite': {'profile': [['NA']]}}}, 'contributors': {'advisor': {'Landau, Luiz': {'profile': [['NA']]}}, 'referee': {'Alves, José Luis Drummond': {'profile': [['NA']]}, 'Marton, Edilson': {'profile': [['NA']]}, 'Cirano, Mauro': {'profile': [['NA']]}, 'Cataldi, Marcio': {'profile': [['NA']]}}}, 'subjectsCNPQ': [['CNPQ::ENGENHARIAS::ENGENHARIA CIVIL']], 'subjectsPOR': [['Oscilações Subinerciais'], ['Ondas de Plataforma'], ['Ondas Con nadas Costeiras'], ['Maré Meteorológica'], ['Frentes Frias'], ['Fluxos de Calor'], ['Plataforma Continental Sudeste do Brasil']], 'institutions': ['UFRJ'], 'departements': ['Instituto Alberto Luiz Coimbra de Pós-Graduação e Pesquisa de Engenharia'], 'programs': ['Programa de Pós-Graduação em Engenharia Civil'], 't